# Building a Variational Autoencoder

We’ve explored the UC Berkeley milling data set – now it’s time for us to build some models. We’ll use a variant of the autoencoder – a variational autoencoder (VAE) – to conduct the anomaly detection.

## Setup Notebook

You'll need python 3.6+ with the following packages in your local environment:

* Numpy
* SciPy
* Pandas
* TensorFlow 2.0

First, we will load all the neccessary packages.

In [1]:
import numpy as np
import scipy.io as sio # for reading matlab files
import pathlib
from pathlib import Path
import pandas as pd
import os
import zipfile

%matplotlib inline

Set the appropriate working folders.

In [12]:
# make your root_dir the current working directory
root_dir = Path.cwd() # set the root directory as a Pathlib path

folder_raw_data = root_dir / 'data/raw/' # raw data folder that holds the .zip .mat files for milling data
folder_processed_data = root_dir / 'data/processed/' # processed data folder

# location to save trained models
folder_models = folder_processed_data / 'models'

We now need to prepare the notebook by downloading the milling data file and other important files. This needs to be done if you are running in google colab. If the repository has been cloned from github, then there is no need.

In [3]:
# extract mill.mat from the zip file
with zipfile.ZipFile(folder_raw_data / 'mill.zip', 'r') as zip_ref:
    zip_ref.extractall(folder_raw_data)

# The Variational Autoencoder 

The variational autoencoder was introduced in 2013 and today is widely used in machine learning applications. The VAE is different from traditional autoencoders in that the VAE is both probabilistic and generative. What does that mean? Well, the VAE creates outputs that are partly random (even after training) and can also generate new data that is like the data it is trained on. 

There are excellent explanations of the VAE online – I'll direct you to Alfredo Canziani’s deep learning course (from [his Youtube](https://www.youtube.com/embed/7Rb4s9wNOmc)). But here is my attempt at an explanation: 


At a high level, the VAE has a similar structure to a traditional autoencoder. However, the encoder learns different codings; namely, the VAE learns mean codings, $\boldsymbol{\mu}$, and standard deviation codings, $\boldsymbol{\sigma}$. The VAE then samples randomly from a Gaussian distribution, with the same mean and standard deviation created by the encoder, to generate the latent variables, $\boldsymbol{z}$. These latent variables are then “decoded” to reconstruct the input. The figure below demonstrates how a signal is reconstructed using the VAE. 

<div style="text-align: center; ">
<figure>
  <img src="images/vae.svg" alt="variational autoencoder process" style="background:none; border:none; box-shadow:none; text-align:center" width="800px"/>
  <div style="text-align: left; ">
  <figcaption style="color:grey; font-size:smaller">A variational autoencoder architecture (top), and an example of a data sample going through the VAE (bottom). Data is compressed in the encoder to create mean and standard deviation codings. The coding, $\boldsymbol{z}$, is then created, with the addition of Gaussian noise, from the mean and standard deviation codings. The decoder uses the codings (or latent variables) $\boldsymbol{z}$ to reconstruct the input. (Image from author, based on graphic from Aurélien Geron)</figcaption>
  </div>
</figure>
</div>

During training, the VAE works to minimize its reconstruction loss, and at the same time, force a Gaussian structure using a latent loss. The structure is achieved through the Kullback-Leibler (KL) divergence, with detailed derivations for the losses in the original VAE paper. Together, the reconstruction loss, and latent loss are as follows: 

  


$$\mathcal{L}(\boldsymbol{x},\boldsymbol{x'}) = \left \| \boldsymbol{x}-\boldsymbol{x'} \right \|^{2} + \frac{\beta}{2}\sum_{i=1}^{K}( \sigma_i - \log(\sigma_i) -1 +  \mu_i^2)$$


  

where $K$ is the number of latent variables, and $\beta$ is an adjustable hyper-parameter introduced by Higgens et al.

A VAE learns factors, embedded in the codings, that can be used to generate new data. As an example of these factors, a VAE may be trained to recognize shapes in an image. One factor may encode information on how pointy the shape is, while another factor may look at how round it is. However, in a VAE, the factors are often entangled together across the codings (the latent variables). Tuning the hyper-parameter $\beta$, to a value larger than one, can enable the factors to disentangle such that each coding only represents one factor at a time. Thus, greater interpretability of the model can be obtained. A VAE with a tunable beta is often called a disentangled-variational-autoencoder, or simply, a $\beta$-VAE. 

# Data Preparation

Before going any further, we need to prepare the data. Ultimately, we'll be using the VAE to detect "abnormal" tool conditions, which correspond to when the tool is in a worn. But first we need to label the data.

As shown in the last post, each cut has an associated amount of wear, measured at the end of the cut. We'll label each cut as either healthy, degraded, or failed according to the amount of wear on the tool. Here's the schema:

| State    | Label | Flank Wear (mm) |
| -------- | ----: | --------------: |
| Healthy  |     0 |         0 ~ 0.2 |
| Degraded |     1 |       0.2 ~ 0.7 |
| Failed   |     2 |           > 0.7 |

I've created a data prep class that takes the raw matlab files, a labelled CSV (each cut is labelled with the associated flank wear), and spits out the training/validation/and testing data (you can see the entire `data_prep.py` in the github repository). However, I want to highlight one function in the class that is important; that is, the create_tensor function.

In [4]:
def create_tensor(self, data_sample, signal_names, start, end, window_size, stride=8):
    """Create a tensor from a cut sample. Final tensor will have shape: 
    [# samples, # sample len, # features/sample]

    Parameters
    ===========
    data_sample : ndarray
        single data sample (individual cut) containing all the signals

    signal_names : tuple
        tuple of all the signals that will be added into the tensor

    start : int
        starting index (e.g. the first 2000 "samples" in the cut may be from
        when the tool is not up to speed, so should be ignored)

    end : int
        ending index

    window_size : int
        size of the window to be used to make the sub-cuts

    stride : int
        length to move the window at each iteration

    Returns
    ===========
    c : ndarray
        array of the sub-cuts
    """

    s = signal_names[::-1]  # only include the six signals, and reverse order
    c = data_sample[s[0]].reshape((9000, 1))

    for i in range(len(s)):
        try:
            a = data_sample[s[i + 1]].reshape((9000, 1))  # reshape to make sure
            c = np.hstack((a, c))  # horizontal stack
        except:
            # reshape into [# samples, # sample len, # features/sample]
            c = c[start:end]
            c = np.reshape(c, (c.shape[0], -1))

    dummy_array = []
    # fit the strided windows into the dummy_array until the length
    # of the window does not equal the proper length
    for i in range(c.shape[0]):
        windowed_signal = c[i * stride : i * stride + window_size]
        if windowed_signal.shape == (window_size, 6):
            dummy_array.append(windowed_signal)
        else:
            break

    c = np.array(dummy_array)
    return c

The `create_tensor` function takes an individual cut, breaks it up into chunks, and puts them into a single array. It breaks the cut signal up into chunks using a window of a fixed size (the window_size variable) and then "slides" the window along the signal. The window "slides" by a preditermined amount, set by the stride variable.

We'll take each of the 165 cuts (remember, two cuts from the original 167 are corrupted) and apply a window size of 64 and a stride of 64 (no overlap between windows). I've visually inspected each cut and selected when the "stable" cutting region occurs, which is usually five seconds or so after the signal begins collection, and a few seconds before the signal collection ends. This information is stored in the "labels_with_tool_class.csv" file.

With all that, we can then create the training/validation/testing data sets. Here is what the script looks like:

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
import data_prep

data_file = folder_raw_data / "mill.mat"


# helper functions needed in the data processing
def scaler(x, min_val_array, max_val_array):
    '''Scale an array across all dimensions'''
    
    # get the shape of the array
    s, _, sub_s = np.shape(x)
    
    for i in range(s):
        for j in range(sub_s):
            x[i,:,j] = np.divide((x[i,:,j] - min_val_array[j]), np.abs(max_val_array[j] - min_val_array[j]))
           
    return x

# min-max function
def get_min_max(x):
    '''Get the minimum and maximum values for each
        dimension in an array'''
    
    # flatten the input array http://bit.ly/2MQuXZd
    flat_vector = np.concatenate(x)

    min_vals = np.min(flat_vector,axis=0)
    max_vals = np.max(flat_vector,axis=0)

    return min_vals, max_vals
    
# use the DataPrep module to load the data
prep = data_prep.DataPrep(data_file)

# load the labeled CSV (NaNs filled in by hand)
df_labels = pd.read_csv(folder_processed_data / "labels_with_tool_class.csv")


# Save regular data set. The X_train, X_val, X_test will be used for anomaly detection
# discard certain cuts as they are strange
cuts_remove = [17, 94]
df_labels.drop(cuts_remove, inplace=True)

# use the return_xy function to take the milling data, select the stable cutting region,
# and apply the window/stride
X, y = prep.return_xy(df_labels, prep.data,prep.field_names[7:],window_size=64, stride=64)

# use sklearn train_test_split function
# use stratify to ensure that the distribution of classes is equal
# between each of the data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=15, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=10, stratify=y_test)

# get the min/max values from each dim in the X_train
min_vals, max_vals = get_min_max(X_train)

# scale the train/val/test with the above min/max values
X_train = scaler(X_train, min_vals, max_vals)
X_val = scaler(X_val, min_vals, max_vals)
X_test = scaler(X_test, min_vals, max_vals)

# training/validation of VAE is only done on healthy (class 0)
# data samples, so we must remove classes 1, 2 from the train/val
# data sets
X_train_slim, y_train_slim = prep.remove_classes(
    [1,2], y_train, X_train
)


X_val_slim, y_val_slim = prep.remove_classes(
    [1,2], y_val, X_val
)

print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape, end='\n\n')

# function to show the percentage of labels in y-labels
def y_shape_percentage(y_train, label):
    
    l = y_train
    
    print('shape {}:'.format(label), l.shape, 
          '\t\t0: {:.1%}'.format(len(l[l == 0])/len(l)), 
          '\t\t1: {:.1%}'.format(len(l[l == 1])/len(l)), 
          '\t\t2: {:.1%}'.format(len(l[l == 2])/len(l)))
    
# let's see what percentage of the data set splits are made up of healthy (0), degraded (1),
# and failed (2) labels
y_shape_percentage(y_train, 'y_train')
y_shape_percentage(y_val, 'y_val')
y_shape_percentage(y_test, 'y_test')


Shape of X_train: (7751, 64, 6)
Shape of y_train: (7751,)

shape y_train: (7751,) 		0: 36.5% 		1: 56.2% 		2: 7.3%
shape y_val: (1909,) 		0: 36.5% 		1: 56.2% 		2: 7.3%
shape y_test: (1910,) 		0: 36.5% 		1: 56.2% 		2: 7.3%


The final distribution of the data is shown below. We'll be training the VAE on only the healthy data (class 0). However, checking the perfomance anomaly detection will be done using all the data. In other words, we'll be training our VAE on the "slim" data sets.

| Data Split      | No. Sub-Cuts | Healthy % | Degraded % | Failed % |
| --------------- | :----------: | :-------: | :--------: | :------: |
| Training full   |    11,570    |   36.5    |    56.2    |   7.3    |
| Training slim   |    2,831     |    100    |     -      |    -     |
| Validation full |    1,909     |   36.5    |    56.2    |   7.3    |
| Validation slim |     697      |    100    |     -      |    -     |
| Testing full    |    1,910     |   36.5    |    56.2    |   7.3    |

# Building the Model

We now understand what a variational autoencoder is and how the data is prepared. Time to build! 

Our VAE will be made up of layers consisting of convolutions layers, batch normalization layers, and max pooling layers. The figure below shows what one of our VAE models could look like. 

<div style="text-align: center; ">
<figure>
  <img src="images/model_architecture.svg" alt="variational autoencoder process" style="background:none; border:none; box-shadow:none; text-align:center"/>
  <div style="text-align: left; ">
  <figcaption style="color:grey; font-size:smaller">Example model architecture used in the $\beta$-VAE. The input to the encoder is a milling data sample, with a window size of 64 for an input shape of [64, 6]. There are 3 convolutional layers, a filter size of 17, and a coding size of 18. (Image from author)</figcaption>
  </div>
</figure>
</div>

I won't be going through all the details of the model -- you can look at the Jupyter notebook and read my code comments. However, here are some important points:

* I've used the temporal convolutional network as the basis for the convolutional layers. The implementation is from Philippe Remy -- thanks Philippe! You can find his github repo [here](https://github.com/philipperemy/keras-tcn).
* Aurélien Geron's book, ["Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow"](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646/ref=sr_1_1?dchild=1&keywords=Aur%C3%A9lien+Geron&qid=1614346360&sr=8-1), is great. In particular, his section on VAEs was incredibly helpful, and I've used some of his methods here. There is a Jupyter notebook from that section of his book on [his github](https://github.com/ageron/handson-ml2/blob/master/17_autoencoders_and_gans.ipynb). Thanks Aurélien!
* I've used rounded accuracy to measure how the model performs during training, as suggested by Geron.

Here is, roughly, what the model function looks like. Note, I've also included the Sampling class and rounded accuracy function.

In [6]:
import tensorflow as tf
from tensorflow import keras
import tensorboard
from tcn import TCN
import numpy as np
import datetime
import traceback

# see what version of tensorflow is installed
print('TensorFlow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('Tensorboard version:', tensorboard.__version__)

TensorFlow version:  2.1.0
Keras version:  2.2.4-tf
Tensorboard version: 2.1.0


In [7]:
# see if you're using a GPU. You don't have to, but it
# will be faster if you're training many models
try:
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))
except:
    print('GPU device not found')

Found GPU at: /device:GPU:0


In [8]:
# build custom sampling function
# Sampling and rounded_accuracy code modified from Aurelion Geron, 
# https://github.com/ageron/handson-ml2/blob/master/17_autoencoders_and_gans.ipynb
# used under Apache 2.0 License, https://github.com/ageron/handson-ml2/blob/master/LICENSE

K = keras.backend

# class for sampling embeddings in the latent space
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean


# rounded accuracy for the metric
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))


# fit the model
def model_fit(
    X_train_slim,
    X_val_slim,
    beta_value=1.25,
    codings_size=10,
    dilations=[1, 2, 4],
    conv_layers=1,
    seed=31,
    start_filter_no=32,
    kernel_size_1=2,
    epochs=10,
    earlystop_patience=8,
    verbose=0,
    compile_model_only=False,
):

    _, window_size, feat = X_train_slim.shape


    # save the time model training began
    # this way we can identify trained model at the end
    date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    # set random seeds so we can somewhat reproduce results
    tf.random.set_seed(seed)
    np.random.seed(seed)

    end_filter_no = start_filter_no

    inputs = keras.layers.Input(shape=[window_size, feat])
    z = inputs

    #### ENCODER ####
    for i in range(0, conv_layers):
        z = TCN(
            nb_filters=start_filter_no,
            kernel_size=kernel_size_1,
            nb_stacks=1,
            dilations=dilations,
            padding="causal",
            use_skip_connections=True,
            dropout_rate=0.0,
            return_sequences=True,
            activation="selu",
            kernel_initializer="he_normal",
            use_batch_norm=False,
            use_layer_norm=False,
        )(z)

        z = keras.layers.BatchNormalization()(z)
        z = keras.layers.MaxPool1D(pool_size=2)(z)

    z = keras.layers.Flatten()(z)
    print("Shape of Z:", z.shape)

    codings_mean = keras.layers.Dense(codings_size)(z)

    codings_log_var = keras.layers.Dense(codings_size)(z)

    codings = Sampling()([codings_mean, codings_log_var])

    variational_encoder = keras.models.Model(
        inputs=[inputs], outputs=[codings_mean, codings_log_var, codings]
    )

    #### DECODER ####
    decoder_inputs = keras.layers.Input(shape=[codings_size])

    x = keras.layers.Dense(
        start_filter_no * int((window_size / (2 ** conv_layers))), activation="selu"
    )(decoder_inputs)

    x = keras.layers.Reshape(
        target_shape=((int(window_size / (2 ** conv_layers))), end_filter_no)
    )(x)

    for i in range(0, conv_layers):
        x = keras.layers.UpSampling1D(size=2)(x)
        x = keras.layers.BatchNormalization()(x)

        x = TCN(
            nb_filters=start_filter_no,
            kernel_size=kernel_size_1,
            nb_stacks=1,
            dilations=dilations,
            padding="causal",
            use_skip_connections=True,
            dropout_rate=0.0,
            return_sequences=True,
            activation="selu",
            kernel_initializer="he_normal",
            use_batch_norm=False,
            use_layer_norm=False,
        )(x)

    outputs = keras.layers.Conv1D(
        feat, kernel_size=kernel_size_1, padding="same", activation="sigmoid"
    )(x)
    variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[outputs])

    _, _, codings = variational_encoder(inputs)
    reconstructions = variational_decoder(codings)
    variational_ae_beta = keras.models.Model(inputs=[inputs], outputs=[reconstructions])

    latent_loss = (
        -0.5
        * beta_value
        * K.sum(
            1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
            axis=-1,
        )
    )

    variational_ae_beta.add_loss(K.mean(latent_loss) / (window_size * feat))
    variational_ae_beta.compile(
        loss="binary_crossentropy",
        optimizer="adam",  #'rmsprop'
        metrics=[rounded_accuracy],
    )

    # count the number of parameters so we can look at this later
    # when evaluating models
    param_size = "{:0.2e}".format(
        variational_encoder.count_params() + variational_decoder.count_params()
    )


    # Model Name
    # b : beta value used in model
    # c : number of codings -- latent variables
    # l : numer of convolutional layers in encoder (also decoder)
    # f1 : the starting number of filters in the first convolution
    # k1 : kernel size for the first convolution
    # k2 : kernel size for the second convolution
    # d : whether dropout is used when sampling the latent space (either True or False)
    # p : number of parameters in the model (encoder + decoder params)
    # eps : number of epochs
    # pat : patience stopping number

    model_name = (
        "TBVAE-{}:_b={:.2f}_c={}_l={}_f1={}_k1={}_dil={}"
        "_p={}_eps={}_pat={}".format(
            date_time,
            beta_value,
            codings_size,
            conv_layers,
            start_filter_no,
            kernel_size_1,
            dilations,
            param_size,
            epochs,
            earlystop_patience,
        )
    )

    print("\n", model_name, "\n")

    if compile_model_only == False:

        earlystop_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0.0002,
            patience=earlystop_patience,
            restore_best_weights=True,
            verbose=1,
        )

        history = variational_ae_beta.fit(
            X_train_slim,
            X_train_slim,
            epochs=epochs,
            batch_size=1024,
            shuffle=True,
            validation_data=(X_val_slim, X_val_slim),
            callbacks=[earlystop_callback,], 
            verbose=verbose,
        )

        return date_time, model_name, history, variational_ae_beta, variational_encoder

    else:

        return variational_ae_beta, variational_encoder

# Training the Model
Time to begin training some models. To select the hyperparameters, we'll be using a random search. Why a random search? Well, it's fairly simple to implement and has been shown to yield better results when compared to a grid search. Scikit-learn has some nice methods for implementing a random search, and that's what we'll use.

We'll be training a bunch of different VAEs, all with different parameters. After each VAE has been trained (trained to minimize reconstruction loss) and the model saved, we'll go through the VAE model and see how it performs in anomaly detection. Here's a diagram of the random search training process:

<div style="text-align: center; ">
<figure>
  <img src="images/vae_training_random_search.png" alt="random search training process" style="background:none; border:none; box-shadow:none; text-align:center"/>
  <div style="text-align: left; ">
  <figcaption style="color:grey; font-size:smaller">The random search training process has three steps. First, randomly select the hyperparameters. Second, train the VAE with these parameters. Third, check the anomaly detection performance of the trained VAE.  (Image from author)</figcaption>
  </div>
</figure>
</div>

In practice, when I ran this experiment, I trained about 1000 VAE models on Google Colab (yay free GPUs!). After all 1000 models were trained, I moved them to my local computer, with a less powerful GPU, and then checked the models for anomaly detection performance. Google Colab has access to great GPUs, but your time with them is limited, so it makes sense to maximize the use of the GPUs on them this way. I'll detail how the trained models are checked for anomaly detection performance in the next blog post.

In [9]:
from sklearn.utils import shuffle
from sklearn.model_selection import ParameterSampler
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

In [10]:
# Input the number of iterations you want to search over
random_search_iterations = 1

# random seed value from system input
ransdom_seed_input = 135

# parameters for beta-vae
p_bvae_grid = {
    "beta_value": uniform(loc=0.5, scale=9),
    "codings_size": sp_randint(5, 40),
    "conv_layers": [3, 2, 1],
    "start_filter_no": sp_randint(16, 128),
    "dilations": [[1, 2, 4, 8], [1, 2, 4], [1, 2]],
    "kernel_size_1": sp_randint(2, 9),
    "earlystop_patience": sp_randint(30, 50),
}

# epochs
epochs = 1

In [13]:
# folder to save models in
model_save_folder = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_mill'

# create the folder
(folder_models / 'saved_models' / model_save_folder).mkdir(parents=True, exist_ok=True)

# create dataframe to store all the results
df_all = pd.DataFrame()

In [14]:
# setup parameters to sample
rng = np.random.RandomState(ransdom_seed_input)

# list of parameters in random search
p_bvae = list(ParameterSampler(p_bvae_grid, n_iter=random_search_iterations,random_state=rng))


for i, params in enumerate(p_bvae):
    print('\n### Run no.', i+1)
    
    ### TRY MODELS ###

    # BETA-VAE
    # parameters  
    beta_value =params["beta_value"]
    codings_size =params["codings_size"]
    conv_layers =params["conv_layers"]   
    start_filter_no =params["start_filter_no"] 
    kernel_size_1 = params["kernel_size_1"]
    dilations = params["dilations"]
    earlystop_patience=params["earlystop_patience"]
    

    seed = 16
    verbose = 1

    # try the model and if it doesn't work, go onto the next model
    # not always the best to use 'try' but good enough
    try:

        date_time, model_name, history, beta_vae_model, bvae_encoder = model_fit(
            X_train_slim,
            X_val_slim,
            beta_value=beta_value,
            codings_size=codings_size,
            conv_layers=conv_layers,
            seed=seed,
            start_filter_no=start_filter_no,
            kernel_size_1=kernel_size_1,
            dilations=dilations,
            epochs=epochs,
            earlystop_patience=earlystop_patience,
            verbose=verbose,
        )

        # save the model. How to: https://www.tensorflow.org/tutorials/keras/save_and_load
        # save model weights and model json
        model_save_dir_bvae = (
            folder_models / 'saved_models' / model_save_folder / (date_time + "_bvae")
        )
        model_save_dir_encoder = (
            folder_models / 'saved_models' / model_save_folder / (date_time + "_encoder")
        )

        # create the save paths
        Path(model_save_dir_bvae).mkdir(parents=True, exist_ok=True)
        Path(model_save_dir_encoder).mkdir(parents=True, exist_ok=True)

        # save entire bvae model
        model_as_json = beta_vae_model.to_json()
        with open(r"{}/model.json".format(str(model_save_dir_bvae)), "w",) as json_file:
            json_file.write(model_as_json)
        beta_vae_model.save_weights(str(model_save_dir_encoder) + "/weights.h5")

        # save encoder bvae model
        model_as_json = bvae_encoder.to_json()
        with open(r"{}/model.json".format(str(model_save_dir_encoder)), "w",) as json_file:
            json_file.write(model_as_json)
        bvae_encoder.save_weights(str(model_save_dir_encoder) + "/weights.h5")

        # get the model run history
        results = pd.DataFrame(history.history)
        epochs_trained = len(results)
        results["epochs_trained"] = epochs_trained
        results = list(
            results[results["val_loss"] == results["val_loss"].min()].to_numpy()
        )  # only keep the top result, that is, the lowest val_loss

        # append best result onto df_model_results dataframe
        if i == 0:
            cols = (
                list(p_bvae[0].keys())
                + list(history.history.keys())
                + ["epochs_trained"]
            )
            results = [[p_bvae[i][k] for k in p_bvae[i]] + list(results[0])]

        else:
            # create dataframe to store best result from model training
            cols = (
                list(p_bvae[0].keys())
                + list(history.history.keys())
                + ["epochs_trained"]
            )
            results = [[p_bvae[i][k] for k in p_bvae[i]] + list(results[0])]

        df = pd.DataFrame(results, columns=cols)

        df["date_time"] = date_time
        df["model_name"] = model_name

        df_all = df_all.append(df, sort=False)

        df_all.to_csv("results_interim_{}.csv".format(model_save_folder))

    except Exception as e:
        print(e)
        print("TRACEBACK")
        traceback.print_exc()
        pass


### Run no. 1
Shape of Z: (None, 224)

 TBVAE-20210430-133238:_b=6.45_c=32_l=3_f1=28_k1=8_dil=[1, 2, 4]_p=2.65e+05_eps=1_pat=40 

Train on 2831 samples, validate on 697 samples
2831/2831 [==============================] - 16s 6ms/sample - loss: 1.8101 - rounded_accuracy: 0.7496 - val_loss: 525.0301 - val_rounded_accuracy: 0.4986


# Conclusion
We've learned about the VAE; prepared the milling data; and implemented the training of the VAE. Phew! Questions?